# ClickHouse. Аналитические функции

In [1]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('/content/drive/MyDrive/dataset/clickhouse_data.csv', sep=';', parse_dates=['dt'])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   dt       49 non-null     datetime64[ns]
 1   group    49 non-null     object        
 2   manager  49 non-null     object        
 3   val      49 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 1.7+ KB


In [9]:
%%capture
!sudo apt-get install apt-transport-https ca-certificates dirmngr
!sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv E0C56BD4

!echo "deb https://repo.clickhouse.tech/deb/stable/ main/" | sudo tee \
    /etc/apt/sources.list.d/clickhouse.list
!sudo apt-get update

!sudo apt-get install -y clickhouse-server clickhouse-client

!sudo service clickhouse-server start
!clickhouse-client

In [10]:
%%capture
!pip install clickhouse-driver

In [11]:
from clickhouse_driver import Client
client = Client(host='localhost')
client.execute('SHOW DATABASES')

[('default',), ('system',)]

In [12]:
client.execute('DROP DATABASE IF EXISTS db')
client.execute('CREATE DATABASE db')
client.execute('SHOW DATABASES')

[('db',), ('default',), ('system',)]

In [13]:
client = Client(host='localhost', user='default', port='9000', database='db')

In [14]:
client.execute('DROP TABLE IF EXISTS sales')
client.execute('CREATE TABLE sales (dt Date, \
                                    group String, \
                                    manager String, \
                                    val  Int32 \
                                      ) ENGINE = Memory')
client.execute('SHOW TABLES FROM db')

[('sales',)]

In [15]:
client.execute("INSERT INTO sales VALUES", df.to_dict('records'))

49

In [16]:
def select_clickhouse(sql):
  return client.query_dataframe(sql)

In [37]:
sql = '''SELECT * FROM sales'''

In [38]:
select_clickhouse(sql)

,dt,group,manager,val
0,2021-01-01,C,m4,95
1,2021-02-01,A,m4,33
2,2021-03-01,C,m4,64
3,2021-04-01,C,m3,1
4,2021-05-01,C,m2,33
5,2021-06-01,B,m1,36
6,2021-07-01,B,m1,35
7,2021-08-01,B,m1,79
8,2021-09-01,A,m5,97
9,2021-10-01,A,m3,10


### Комбинаторы агрегатных функций

If К имени любой агрегатной функции может быть приписан суффикс -If. В этом случае, агрегатная функция принимает ещё один дополнительный аргумент - условие (типа UInt8). Агрегатная функция будет обрабатывать только те строки, для которых условие сработало. Если условие ни разу не сработало - возвращается некоторое значение по умолчанию (обычно - нули, пустые строки).

Примеры: sumIf(column, cond), countIf(cond), avgIf(x, cond), quantilesTimingIf(level1, level2)(x, cond), argMinIf(arg, val, cond) и т. п.

In [29]:
sql = '''select s.group, 
                sumIf(s.val,s.manager='m1') as m1,
                sumIf(s.val,s.manager='m2') as m2,
                sumIf(s.val,s.manager='m3') as m3,
                sumIf(s.val,s.manager='m4') as m4,
                sumIf(s.val,s.manager='m5') as m5
         from sales as s
         group by s.group'''

In [30]:
select_clickhouse(sql)

,group,m1,m2,m3,m4,m5
0,B,183,173,206,177,8
1,C,78,337,100,259,86
2,A,130,82,38,111,170


### Параметрические агрегатные функции

sequenceCount(pattern)(time, cond1, cond2, …)
Вычисляет количество цепочек событий, соответствующих шаблону. Функция обнаруживает только непересекающиеся цепочки событий. Она начинает искать следующую цепочку только после того, как полностью совпала текущая цепочка событий.

In [105]:
sql = '''SELECT sequenceCount('(?1)(?t==1)(?2)(?t==1)(?3)')(s.dt, s.group = 'A',s.group = 'A',s.group='A') as A_A_A 
        FROM sales as s'''

In [106]:
select_clickhouse(sql)

,A_A_A
0,1
